# PyBADS Example 1: Basic usage

In this introductory example, we will show a simple usage of Bayesian Adaptive Direct Search (BADS) to perform optimization of a synthetic target function.

This notebook is Part 1 of a series of notebooks in which we present various example usages for BADS with the PyBADS package.

In [1]:
import numpy as np
from pybads.bads.bads import BADS

## 0. What is (Py)BADS?

BADS is a fast hybrid Bayesian optimization algorithm designed to solve difficult optimization problems, in particular related to parameter estimation -- aka model fitting -- of computational models (e.g., via maximum-likelihood or maximum-a-posteriori estimation). **PyBADS is its Python implementation**.

BADS has been intensively tested for fitting a variety of computational models, and is currently used by many research groups around the world (see [Google Scholar](https://scholar.google.co.uk/scholar?cites=7209174494000095753&as_sdt=2005&sciodt=0,5&hl=en) for many example applications). In our benchmark with real model-fitting problems, BADS performed on par or better than many other common and state-of-the-art optimizers, as shown in the [original BADS paper](https://arxiv.org/abs/1705.04405).

BADS is recommended when no gradient information is available, and the objective function is non-analytical or noisy, for example evaluated through numerical approximation or via simulation.
It requires no specific tuning and runs off-the-shelf like other built-in optimizers (e.g., from `scipy.optimize.minimize`).

*Note*: If you are interested in estimating posterior distributions (i.e., uncertainty and error bars) over model parameters, and not just point estimates, you might also want to check out Variational Bayesian Monte Carlo for Python (PyVBMC), a package for Bayesian posterior and model inference which can be used in synergy with PyBADS.

## 1. Problem definition

Here we show PyBADS at work on [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) in 2D as target function.  
Like most optimizers, PyBADS aims to solve a *minimization* problem.

We specify wide hard bounds and tighter plausible bounds that (hopefully) contain the solution. 
- Hard lower/upper bounds `lb`, `ub` are the actual optimization bounds; PyBADS will not evaluate the target function outside these bounds, but might evaluate them on the bounds. You can use `-np.inf` and `np.inf` for unbounded parameters.
- Plausible lower/upper bounds `plb`, `pub` represent our best guess at bounding the region where the solution might lie. The plausible bounds do not change the optimization problem, but help define starting points and hyperparameters of PyBADS.

We set a starting point for the optimization $\mathbf{x}_0 = (0, 0)$.

In [2]:
def rosenbrocks_fcn(x):
    """Rosenbrock's 'banana' function in any dimension."""
    x_2d = np.atleast_2d(x)
    return np.sum(100 * (x_2d[:, 0:-1]**2 - x_2d[:, 1:])**2 + (x_2d[:, 0:-1]-1)**2, axis=1)

target = rosenbrocks_fcn;

lb = np.array([[-20, -20]])     # Lower bounds
ub = np.array([[20, 20]])       # Upper bounds
plb = np.array([[-5, -5]])      # Plausible lower bounds
pub = np.array([[5, 5]])        # Plausible upper bounds
x0 = np.array([[0, 0]]);        # Starting point

## 2. Initialize and run the optimization

Then, we initialize a `bads` instance which takes care of the optimization. For now, we use default options.  
To run the optimization, we simply call `bads.optimize()`.

In [3]:
bads = BADS(target, x0, lb, ub, plb, pub)
x_min, fval = bads.optimize()

Variables (index) defined with periodic boundaries: []
Beginning optimization of a DETERMINISTIC objective function

 Iteration f-count     f(x)     MeshScale     Method     Actions
     0         3       1.000000      1.000000            Uncertainty test
     0         7       1.000000      1.000000     Initial mesh       Initial points


c:\users\luigi\documents\github\gpyreg\gpyreg\covariance_functions.py:376: RuntimeWarning: invalid value encountered in add
  plausible_lower_bounds[i_nan] + plausible_upper_bounds[i_nan]


     0        11       1.000000      0.500000     Refine grid       Train
     1        19       1.000000      0.250000     Refine grid       Train
     2        21       0.304130      0.250000     Successful search (('ES-wcm', 1))       
     2        23       0.090010      0.250000     Successful search (('ES-ell', 1))       
     2        24       0.022646      0.250000     Incremental search (('ES-ell', 1))       
     2        27       0.000208      0.250000     Incremental search (('ES-ell', 1))       
     2        31       0.000208      0.125000     Refine grid       Train
     3        33       0.000130      0.125000     Incremental search (('ES-ell', 1))       
     3        39       0.000130      0.062500     Refine grid       
bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
     4        47       0.000130      0.031250     Refine grid       
     5        48       0.000082      0.031250     Incremental search (('ES-wcm', 1))   

c:\users\luigi\documents\github\pybads\pybads\bads\bads.py:1760: RuntimeWarning: divide by zero encountered in true_divide
  zscore = zscore / gp_ys
c:\users\luigi\documents\github\pybads\pybads\bads\bads.py:1555: RuntimeWarning: divide by zero encountered in true_divide
  gamma_z = (self.optim_state['f_target'] - self.sufficient_improvement - f_mu) / fs


bads: The optimization is stalling, decreasing further the mesh size
     5        55       0.000082      0.007812     Refine grid       Train
     6        57       0.000040      0.007812     Incremental search (('ES-wcm', 1))       
bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
bads: The optimization is stalling, decreasing further the mesh size
     6        63       0.000040      0.001953     Refine grid       Train


ZeroDivisionError: float division by zero

## 3. Results and conclusions

In [ ]:
print(f"BADS minimum at: x = {x_min.flatten()}, fval = {fval:.4g}")
print(f"total f-count: {bads.function_logger.func_count-1}, time: {round(bads.optim_state['total_time'], 2)} s")

The true minimum of the Rosenbrock function is at $\textbf{x}^\star_\text{min} = [1, 1]$, where $f^\star_\text{min} = 0$.  

In conclusion, PyBADS found the solution with a fairly small number of function evaluations (`f-count`), which is particularly important if the target function is mildly-to-very expensive to compute as in many computational models.

*Note*: PyBADS by default does not aim for extreme numerical precision of the target (e.g., beyond the 2nd or 3rd decimal place), since in most realistic model-fitting problems a higher resolution is typically pointless, e.g. due to noise or variability in the data.

## Example 1: Full code

See [here](./src/pybads_example_1_basic_usage.py) for a Python file with the code used in this example, with no extra fluff.